## VAE Hawkes Process Estimation - Tutorial

In [6]:
import os
import sys

import numpy as np
import pandas as pd
import Hawkes as hk
from matplotlib import pyplot as plt

from HAWKES.hawkes import hawkes_simulations, hawkes_simulation
from HAWKES.hyperparameters import hyper_params_simulation
from HAWKES.discretisation import discretise

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Preprocessing

In [7]:
# Training/Validation/Testing dataset generation

# Intensity Decay Parameter (β) = U(p = 1, q = 3)
# Branching Ratio (η) = U(a = 0.05, b = 0.8)
# Expected Activity (E) = 500
# Time Horizon (T) = 100
# Interval Length (∆) = 1
# Number of processes = 160_000

# Hawkes process hyper-parameters generation
train_params, train_alpha, train_beta, train_mu = hyper_params_simulation(filename="train_hawkes_hyperparams.csv")

# Hawkes processes simulations
train_simulated_events_seqs = hawkes_simulations(train_mu, train_alpha, train_beta, filename='train_hawkes_simulations.csv')

# Discrétiser les processus de Hawkes
train_discret_simulated_events_seqs = discretise(train_simulated_events_seqs, filename='train_binned_hawkes_simulations.csv')